In [1]:
import numpy as np
import scipy.stats as ss
import datetime as dt

# Black Scholes option price
# Function approach

In [2]:
# Black-Scholes option price

def bsm_option_price(strike, spot, vol, texp, intr=0.0, divr=0.0, cp_sign=1):
    vol_std = vol * np.sqrt(texp)
    div_fac = np.exp(-texp*divr)
    disc_fac = np.exp(-texp*intr)
    forward = spot / disc_fac * div_fac
    d1 = np.log(forward/strike)/vol_std + 0.5*vol_std
    d2 = d1 - vol_std

    price = cp_sign * disc_fac \
        * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
    return price

### Different ways of using function arguments

In [3]:
### full arguments
c1 = bsm_option_price(105, 100, 0.2, 0.25, 0, 0, 1)

### omit arguments with default vaules
c2 = bsm_option_price(105, 100, 0.2, 0.25)

### you need to pass all required arguments
#c2 = bsm_option_price(105, 100, 0.2)

### use argument names
c3 = bsm_option_price(105, 100, texp=0.25, vol=0.2)

### always put the named args at the end
#c3 = bsm_option_price(105, texp=0.25, 100, vol=0.2)

c1, c2, c3

(2.0640191378988462, 2.0640191378988462, 2.0640191378988462)

## Disadvantage of function approach:
* The computation in the function has to be repeated every time.
* Some part doesn't have to be repeated: for example, __vol, texp, intr, divr__ don't change often. So   __vol_std, disc_fac, div_fac__ don't have to be computed.
* You feel some needs to organize by binding some data + function(method)

# Class approach
We're going to bind __vol, intr, intd__ into a class

In [4]:
class BSM_Ver1:
    vol, texp, intr, divr = None, None, None, None
    
    def __init__(self, vol, texp, intr=0.0, divr=0.0): # Constructor
        self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
    
    def price(self, strike, spot, cp_sign=1):
        # cp_sign = 1 for call, -1 for put
        vol_std = self.vol * np.sqrt(self.texp)
        div_fac = np.exp(-self.texp*self.divr)
        disc_fac = np.exp(-self.texp*self.intr)
        forward = spot / disc_fac * div_fac
        d1 = np.log(forward/strike)/vol_std + 0.5*vol_std
        d2 = d1 - vol_std

        price = cp_sign * disc_fac \
            * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
        return price

In [5]:
### define a bsm model with spot, vol and expiry.
bsm1 = BSM_Ver1(vol=0.2, texp=1, intr=0.0)
print( bsm1.__dict__ )

### price options with strike
bsm1.price(102, 100), bsm1.price(strike=98, spot=100, cp_sign=-1)

{'intr': 0.0, 'divr': 0.0, 'texp': 1, 'vol': 0.2}


(7.0844942478298947, 6.925974960378305)

In [6]:
### Lets change vol/expiry
bsm1.vol = 0.4
#bsm1.texp = 3

### option price changes
bsm1.price(102, 100), bsm1.price(strike=102, spot=100, cp_sign=-1)

(15.029816377425, 17.029816377425)

In [7]:
class BSM_Ver2:
    vol, texp, intr, divr = None, None, None, None

    def __init__(self, vol, texp, intr=0.0, divr=0.0): # Constructor for this class.
        self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
        
        self.vol_std = vol * np.sqrt(texp)
        self.div_fac = np.exp(-texp*divr)
        self.disc_fac = np.exp(-texp*intr)
    
    def price(self, strike, spot, cp_sign=1):
        forward = spot / self.disc_fac * self.div_fac
        d1 = np.log(forward/strike)/self.vol_std + 0.5*self.vol_std
        d2 = d1 - self.vol_std
        price = cp_sign * self.disc_fac \
            * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
        return price

In [8]:
bsm2 = BSM_Ver2(vol=0.2, texp=1)

bsm2.price(102, 100), bsm2.price(102, 100, cp_sign=-1)

(7.0844942478298947, 9.0844942478298805)

In [9]:
### But... now things are not that simple
bsm2.vol = 0.4
### option prices are price same after volatility change !@#$
bsm2.price(102, 100), bsm2.price(102, 100, cp_sign=-1)
#print( bsm2.__dict__ )

(7.0844942478298947, 9.0844942478298805)

In [10]:
class BSM_Ver3:
    vol, texp, intr, divr = None, None, None, None

    def __init__(self, vol, texp, intr=0.0, divr=0.0): # Constructor
        #self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
        self.setparams(vol, texp, intr, divr)

    def setparams(self, vol=None, texp=None, intr=None, divr=None):
        self.vol = vol if(vol != None) else self.vol
        self.texp = texp if(texp != None) else self.texp
        self.intr = intr if(intr != None) else self.intr
        self.divr = divr if(divr != None) else self.divr
        
        self.vol_std = self.vol * np.sqrt(self.texp)
        self.div_fac = np.exp(-self.texp*self.divr)
        self.disc_fac = np.exp(-self.texp*self.intr)
    
    def price(self, strike, spot, cp_sign=1):
        forward = spot / self.disc_fac * self.div_fac
        d1 = np.log(forward/strike)/self.vol_std + 0.5*self.vol_std
        d2 = d1 - self.vol_std
        price = cp_sign * self.disc_fac \
            * ( forward * ss.norm.cdf(cp_sign*d1) - strike * ss.norm.cdf(cp_sign*d2) )
        return price

In [11]:
bsm3 = BSM_Ver3(vol=0.2, texp=1.0)

bsm3.price(102, 100, 1), bsm3.price(102, 100, -1)

(7.0844942478298947, 9.0844942478298805)

In [12]:
### not let's try setparams method
bsm3.setparams(vol=0.4, intr=0.01)
### option prices are price same after volatility change !@#$
bsm3.price(102, 100, 1), bsm3.price(102, 100, -1)

#print( bsm2.__dict__ )

(15.442197598441437, 16.42728064085658)

In [13]:
class OptionContract:
    def __init__(self, undl, opt_type, strike, dexp):
        ''' Constructor for this class. '''
        self.undl, self.strike, self.dexp = undl, strike, dexp
        self.opt_type = opt_type
        self.cp_sign = 1 if (opt_type == 'call') else -1

    def toString(self):
        return('{:s} option on {:s} struck at {:0.1f} maturing on {:s}'\
              .format(self.opt_type, self.undl, self.strike, self.dexp.strftime('%Y.%m.%d')))

    def price(self, spot, model):
        return model.priceFromContract(spot, self)
    
class BSM_Model(BSM_Ver3): #Class Inheritance
    def priceFromContract(self, spot, contract):
        texp = (contract.dexp - dt.date.today()).days/365.25
        if(abs(texp - self.texp)>1e-12):
            print('Resetting texp of model to {:0.3f}'.format(texp))
            self.setparams(texp=texp)
        return self.price(contract.strike, spot, contract.cp_sign)
 

In [14]:
tc_c105_dec = OptionContract('Tencent', 'call', 105, dexp=dt.date(2017, 12, 25))
tc_p95_dec = OptionContract('Tencent', 'put', 95, dexp=dt.date(2017, 12, 25))

print(tc_c105_dec.toString())
print(tc_p95_dec.toString())



call option on Tencent struck at 105.0 maturing on 2017.12.25
put option on Tencent struck at 95.0 maturing on 2017.12.25


In [15]:
bsm_model = BSM_Model(vol=0.2, texp=1)

tc_spot = 100
price1 = tc_c105_dec.price(tc_spot, model=bsm_model)
price2 = tc_p95_dec.price(tc_spot, model=bsm_model)
print( price1, price2 )

### Similarly....
price3 = bsm_model.priceFromContract(tc_spot, tc_c105_dec)
price4 = bsm_model.priceFromContract(tc_spot, tc_p95_dec)
print( price3, price4 )

Resetting texp of model to 0.287
2.3282938521 2.13651991307
2.3282938521 2.13651991307


In [16]:
print(bsm_model.__dict__)

{'vol_std': 0.1072332658551199, 'divr': 0.0, 'texp': 0.2874743326488706, 'vol': 0.2, 'div_fac': 1.0, 'intr': 0.0, 'disc_fac': 1.0}


In [36]:
import random
class Choi_Model:
    def __init__(self):
        pass
    def priceFromContract(self, spot, contract):
        return( (spot-contract.strike) + random.random()*10  )

In [45]:
tc_spot = 100
choi_model = Choi_Model()
price1 = tc_c105_dec.price(tc_spot, model=choi_model)
price2 = tc_p95_dec.price(tc_spot, model=choi_model)
print( price1, price2 )

0.6135636413269276 14.378764950575812
